In [67]:
import numpy as np
import os
from Bio import SeqIO #pip install biopython
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import time

In [2]:
#step 1
def generate_motif(ML, ICPC): 
    if ICPC == 2:
        results = []
        for i in range(ML):
            col = [0,0,0,0]
            loc = np.random.randint(0,4)
            col[loc] = 1
            results.append(col)
        return results
    
    
    eplison = 0.01
    
    def n_sum(ICs, ML, ICPC):
        if ML == 6:
            for a in range(0, len(ICs) - 5):
                for b in range(1, len(ICs) - 4):
                    for c in range(2, len(ICs) - 3):
                        for d in range(3, len(ICs) - 2):
                            for e in range(4, len(ICs) - 1):
                                for f in range(5, len(ICs)):
                                    if abs(ICs[a] + ICs[b] + ICs[c] + ICs[d] + ICs[e] + ICs[f] - ICPC * 6) <  eplison and len(set([a,b,c,d,e,f])) == 6:
                                        return [a,b,c,d,e,f]
        elif ML == 7:
            for a in range(0, len(ICs) - 6):
                for b in range(1, len(ICs) - 5):
                    for c in range(2, len(ICs) - 4):
                        for d in range(3, len(ICs) - 3):
                            for e in range(4, len(ICs) - 2):
                                for f in range(5, len(ICs) - 1):
                                    for g in range(5, len(ICs)):
                                        if abs(ICs[a] + ICs[b] + ICs[c] + ICs[d] + ICs[e] + ICs[f] + ICs[g] - ICPC * 7) <  eplison and len(set([a,b,c,d,e,f,g])) == 7:
                                            return [a,b,c,d,e,f,g]
        else: #8
            for a in range(0, len(ICs) - 7):
                for b in range(1, len(ICs) - 6):
                    for c in range(2, len(ICs) - 5):
                        for d in range(3, len(ICs) - 4):
                            for e in range(4, len(ICs) - 3):
                                for f in range(5, len(ICs) - 2):
                                    for g in range(5, len(ICs) - 1):
                                        for h in range(6, len(ICs)):
                                            if abs(ICs[a] + ICs[b] + ICs[c] + ICs[d] + ICs[e] + ICs[f] + ICs[g] + ICs[h] - ICPC * 8) <  eplison and len(set([a,b,c,d,e,f,g,h])) == 8:
                                                return [a,b,c,d,e,f,g,h]
        return []
    
    def calc_IC(col):
        return sum([col[i]*np.log2(4 * col[i]) for i in range(4)])
    
    
    idxs = []
    while len(idxs) == 0:
        cols = []
        ICs = []
        while len(ICs) < ML * 2:
            rvs = sorted(np.random.uniform(size = 3))
            col = [rvs[0] - 0, rvs[1] - rvs[0], rvs[2] - rvs[1], 1 - rvs[2]]
            IC = calc_IC(col)
            if IC >= ICPC - 0.1:
                ICs.append(IC)
                cols.append(col)
        idxs = n_sum(ICs, ML, ICPC)
    
    results = []
    for idx in idxs:
        results.append(cols[idx])
        
    return results
    
    
def build_benchmark(ICPC, ML, SL, SC, index):
    ACGT = ['A', 'C', 'G', 'T']
    #2)
    sequences = []
    for i in range(SC):
        sequence = []
        for j in range(SL):
            k = np.random.randint(0,4)
            sequence.append(ACGT[k])
        sequences.append(sequence)
    #3)
    motif = generate_motif(ML, ICPC)
    #4)
    sites = []
    for i in range(SC):
        site = []
        for j in range(ML):
            if ICPC == 2:
                if motif[j][0] == 1:
                    site.append(ACGT[0])
                elif motif[j][1] == 1:
                    site.append(ACGT[1])
                elif motif[j][2] == 1:
                    site.append(ACGT[2])
                else:
                    site.append(ACGT[3])
            else:
                k = np.random.uniform()
                if k < motif[j][0]:
                    site.append(ACGT[0])
                elif k >= motif[j][0] and k < motif[j][0] + motif[j][1]:
                    site.append(ACGT[1])
                elif k >= motif[j][0] + motif[j][1] and k < motif[j][0] + motif[j][1] + motif[j][2]:
                    site.append(ACGT[2])
                else:
                    site.append(ACGT[3])
        sites.append(site)
    #5)
    sites_location = []
    for i in range(SC):
        j = np.random.randint(0,SC) #check idx
        k = np.random.randint(0,SL - ML) #check idx
        sequences[i][k: k + ML] = sites[j] #check idx
        sites_location.append(k)
    #6)
    location = 'data_set_' + str(index)
    if not os.path.exists(location):
        os.makedirs(location)
    
    s = []   
    for i in sequences:
        s.append(SeqRecord(Seq(''.join(i))))
    
    SeqIO.write(s, os.path.join(location, 'sequences' + ".fasta"), "fasta")
    
    #7)
    f = open(os.path.join(location, 'sites' + ".txt"), 'w+')
    for i in sites_location:
        f.write(str(i))
        f.write(',')
    f.close()
    #8)
    f = open(os.path.join(location, 'motif' + ".txt"), 'w+')
    f.write('MOTIF' + str(index) + ' ' + str(ML))
    f.write('\n')
    for i in motif:
        for j in i:
            f.write(str(j))
            f.write(',')
        f.write('\n')
    f.close()
    #9)
    with open(os.path.join(location, 'motiflength' + ".txt"), 'w+') as f:
        f.write(str(ML))
        #f.close()

In [3]:
build_benchmark(1,7,500,10,100)
#ACCCATA

[[0.7399634160501809, 0.01671863874975621, 0.22411878707001898, 0.019199158130043914], [0.07518210899784561, 0.8192967346332177, 0.09093449824937727, 0.014586658119559393], [0.01433203576998554, 0.642080628405567, 0.333364465060223, 0.010222870764224457], [0.14111228974756207, 0.7790748212693915, 0.046316047793121484, 0.03349684118992491], [0.8008593715120103, 0.08019829204541207, 0.06850473408611546, 0.05043760235646222], [0.19165227003013052, 0.007651350589455208, 0.044894171360402635, 0.7558022080200116], [0.825596292113333, 0.07806998256756847, 0.08846525886384093, 0.007868466455257628]]
[['G', 'C', 'C', 'C', 'A', 'T', 'A'], ['G', 'C', 'G', 'C', 'A', 'A', 'A'], ['A', 'C', 'C', 'C', 'A', 'T', 'A'], ['A', 'C', 'C', 'T', 'A', 'T', 'A'], ['A', 'C', 'C', 'C', 'A', 'T', 'A'], ['G', 'C', 'C', 'C', 'A', 'T', 'A'], ['A', 'G', 'C', 'C', 'C', 'T', 'A'], ['A', 'C', 'G', 'C', 'C', 'T', 'A'], ['A', 'C', 'C', 'C', 'A', 'A', 'G'], ['G', 'C', 'C', 'G', 'A', 'T', 'A']]


In [22]:
for i in range(10):
    build_benchmark(1,7,500,10,0 + i)

In [51]:
for i in range(10):
    build_benchmark(1.5,7,500,10,10 + i)

In [36]:
for i in range(10):
    build_benchmark(2,7,500,10,20 + i)
for i in range(10):
    build_benchmark(1.5,6,500,10,30 + i)

In [37]:
for i in range(10):
    build_benchmark(1.5,8,500,10,40 + i)

In [38]:
for i in range(10):
    build_benchmark(1.5,7,500,5,50 + i)

In [39]:
for i in range(10):
    build_benchmark(1.5,7,500,20,60 + i)

In [65]:
#step 2
def motif_finder(index):       
    location = 'data_set_' + str(index) + '/' 
    seqs = []
    for record in SeqIO.parse(os.path.join(location, 'sequences' + ".fasta"), "fasta"):
        seqs.append(str(record.seq))

    with open(os.path.join(location,'motiflength.txt'), 'r') as f:
        ML = int(f.read())
    
    def calc_ICs(motif):
        IC = 0
        for col in motif:
            for i in range(4):
                if col[i] != 0:
                    IC += col[i]*np.log2(4 * col[i])
        return IC
    
    def build_motif(seqs):
        counts = np.zeros((len(seqs[0]),4))
        for i in range(len(seqs)):
            for j in range(len(seqs[i])):
                if seqs[i][j] == 'A':
                    counts[j][0] += 1
                elif seqs[i][j] == 'C':
                    counts[j][1] += 1
                elif seqs[i][j] == 'G':
                    counts[j][2] += 1
                elif seqs[i][j] == 'T':
                    counts[j][3] += 1
        motif = np.zeros((len(seqs[0]),4))
        for i in range(len(seqs[0])):
            for j in range(4):
                motif[i][j] = counts[i][j] / sum(counts[i])
        IC = calc_ICs(motif)
        return motif, IC
    

    best_motifs = []
    for s0 in range(len(seqs) - 1):
        for s1 in range(s0 + 1, len(seqs)):
            temp_motifs = []
            for i in range(len(seqs[s0]) - ML):
                for j in range(len(seqs[s0]) - ML):
                    motif, IC = build_motif([seqs[s0][i:i+ML], seqs[s1][j:j+ML]])
                    temp_motifs.append([[i, j], motif, IC, [seqs[s0][i:i+ML], seqs[s1][j:j+ML]]])
                    #if len(best_motif) == 0 or (len(best_motif) > 0 and best_motif[2] < IC):
                        #best_motif = [[i, j], motif, IC, [seqs[s0][i:i+ML], seqs[s1][j:j+ML]]]
            temp_motifs.sort(key = lambda x:-x[2])
            
            s = set(range(len(seqs))) - set([s0,s1])
            
            for idx in range(10):
                best_motif = temp_motifs[idx]            
                for i in s:
                    curr_best_motif = []
                    for j in range(len(seqs[s0]) - ML):
                        motif, IC = build_motif(best_motif[3] + [seqs[i][j:j+ML]])
                        if len(curr_best_motif) == 0 or (len(curr_best_motif) > 0 and curr_best_motif[2] < IC):
                            curr_best_motif = [best_motif[0] + [j], motif, IC, best_motif[3] + [seqs[i][j:j+ML]]]
                    best_motif = curr_best_motif
                best_motifs.append(best_motif)
            
            
    best_motifs.sort(key=lambda x: -x[2])
       
    motif = best_motifs[0][1]
    sites_location = best_motifs[0][0]
    
    location = 'data_set_' + str(index)
    f = open(os.path.join(location, 'predicted_motif' + ".txt"), 'w+')
    f.write('MOTIF' + str(index) + ' ' + str(ML))
    f.write('\n')
    for i in motif:
        for j in i:
            f.write(str(j))
            f.write(',')
        f.write('\n')
    f.close()
    
    f = open(os.path.join(location, 'predicted_sites' + ".txt"), 'w+')
    for i in sites_location:
        f.write(str(i))
        f.write(',')
    f.close() 

In [84]:
#step 3
def evaluation(index, start, end):
    location = 'data_set_' + str(index) + '/' 
    with open(os.path.join(location,'motiflength.txt'), 'r') as f:
        ML = int(f.read())
    with open(os.path.join(location,'motif.txt'), 'r') as f:
        x = f.read().strip().split('\n')
        y = [i.strip().split(',')[:-1] for i in x][1:]
        motif = [[float(j) for j in i] for i in y]
    with open(os.path.join(location,'predicted_motif.txt'), 'r') as f:
        x = f.read().strip().split('\n')
        y = [i.strip().split(',')[:-1] for i in x][1:]
        predicted_motif = [[float(j) for j in i] for i in y]
    with open(os.path.join(location,'sites.txt'), 'r') as f:
        sites = [int(i) for i in f.read().strip().split(',')[:-1]]
    with open(os.path.join(location,'predicted_sites.txt'), 'r') as f:
        predicted_sites = [int(i) for i in f.read().strip().split(',')[:-1]]
    #1)
    def KL(a, b):
        a = np.asarray(a, dtype=np.float)
        b = np.asarray(b, dtype=np.float)
        return np.sum(np.where(a != 0, a * np.log(a / b), 0))

    a = sum([KL(motif[i],predicted_motif[i]) for i in range(len(motif))])
    #2)
    b = 0
    for i in range(len(sites)):
        x = range(sites[i], sites[i] + ML)
        y = range(predicted_sites[i], predicted_sites[i] + ML)
        b += len(set(x) & set(y))
    #3)
    c = sum([1 for i in range(len(sites)) if abs(sites[i] - predicted_sites[i]) < len(motif)/2])
    #4)
    d = end - start
    
    return a, b, c, d

In [20]:
results = []
for i in range(70):
    start = time.time()
    motif_finder(i)
    end = time.time()
    results.append(evaluation(i, start, end))  

C:\Users\ericp\anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\Users\ericp\anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in true_divide


IndexError: list index out of range

In [21]:
for i, r in enumerate(results):
    print("index: %s, kl: %s, overlap pos: %s, overlap site: %s, time: %s sec" 
          %(str(i),str(np.round(r[0], 3)),str(np.round(r[1], 3)),str(np.round(r[2], 3)),str(np.round(r[3], 3))))

index: 0, kl: 5.093, overlap pos: 0, overlap site: 0, time: 11.288 sec
index: 1, kl: 4.756, overlap pos: 0, overlap site: 0, time: 11.223 sec
index: 2, kl: 4.796, overlap pos: 0, overlap site: 0, time: 11.321 sec
index: 3, kl: 4.792, overlap pos: 0, overlap site: 0, time: 11.257 sec
index: 4, kl: 4.962, overlap pos: 0, overlap site: 0, time: 11.232 sec
index: 5, kl: 4.824, overlap pos: 0, overlap site: 0, time: 11.343 sec
index: 6, kl: 4.698, overlap pos: 0, overlap site: 0, time: 11.344 sec
index: 7, kl: 4.784, overlap pos: 0, overlap site: 0, time: 11.351 sec
index: 8, kl: 4.869, overlap pos: 0, overlap site: 0, time: 11.375 sec
index: 9, kl: 4.822, overlap pos: 0, overlap site: 0, time: 11.337 sec
index: 10, kl: 7.224, overlap pos: 0, overlap site: 0, time: 11.366 sec
index: 11, kl: 6.979, overlap pos: 0, overlap site: 0, time: 11.336 sec
index: 12, kl: 5.855, overlap pos: 0, overlap site: 0, time: 11.329 sec
index: 13, kl: 6.196, overlap pos: 1, overlap site: 1, time: 11.627 sec
in

In [68]:
i = 35
start = time.time()
motif_finder(i)
end = time.time()

(inf, 0, 0, 1551.259818315506)


C:\Users\ericp\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in true_divide


In [78]:
print(evaluation(i, start, end))  

4.6508167303512895
3.4599067622405144
2.797371223767228
0.2197595458536939
5.254642560499226
3.1226723495394912
(19.505169172251442, 0, 0, 1551.259818315506)


C:\Users\ericp\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log
C:\Users\ericp\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in multiply
C:\Users\ericp\anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
C:\Users\ericp\anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in multiply


In [80]:
i = 25
start = time.time()
motif_finder(i)
end = time.time()

In [81]:
print(evaluation(i, start, end))  

(0.0, 42, 6, 1782.5443758964539)


C:\Users\ericp\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


In [82]:
i = 45
start = time.time()
motif_finder(i)
end = time.time()

In [85]:
print(evaluation(i, start, end))  

(inf, 48, 6, 1990.7548813819885)


C:\Users\ericp\anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in true_divide


In [93]:
lines = []
with open('Performance Output.txt', 'r') as f:
    for line in f.readlines():
        l = []
        for i in line.split(' '):
            try:
                l.append(float(i.replace(',','')))
            except:
                pass
        lines.append(l)

for i in range(0,70,10):
    s = lines[i:i+10]
    print("---------------------------------")
    print(i)
    for j in range(1,5):
        x = []
        for k in range(10):
            x.append(s[k][j])
        print("-----")
        print(j)
        print(np.mean(x))
        print(np.std(x))
        print("-----")
    print("---------------------------------")

---------------------------------
0
-----
1
22.427200000000003
7.981487340088938
-----
-----
2
11.8
19.928873525616044
-----
-----
3
1.7
2.8653097563788803
-----
-----
4
200.46790000000001
25.952647153806875
-----
---------------------------------
---------------------------------
10
-----
1
12.1396
16.548697484696493
-----
-----
2
37.1
24.65948093533195
-----
-----
3
5.3
3.5227829907617076
-----
-----
4
206.9596
26.815470270722457
-----
---------------------------------
---------------------------------
20
-----
1
0.020999999999999998
3.469446951953614e-18
-----
-----
2
69.3
2.0999999999999996
-----
-----
3
9.9
0.3
-----
-----
4
214.0201
17.84921238850611
-----
---------------------------------
---------------------------------
30
-----
1
21.649700000000003
13.392691708913484
-----
-----
2
17.1
21.983857714241147
-----
-----
3
2.9
3.726929030716845
-----
-----
4
149.09470000000002
15.408582673627059
-----
---------------------------------
---------------------------------
40
-----
1
1